<a href="https://colab.research.google.com/github/s2009457/weakly-crack-detection/blob/main/AshokJinde_Lavanya_Dissertation_Implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

INTRODUCTION : Automated crack identification is a crucial activity that may dramatically minimize manual labor-intensive construction and road inspections. The recent approaches, typically rely significantly on expensive notation procedures. In addition, fresh lots of annotations are generally needed for each new environment in order to accommodate a large range of target domains. During the use of crack detection systems in real life the data annotation costs an important bottleneck. The follwoing implementation of weakly-supervised problem with crack detection offer a 3-branch framework with addition of edge detection implementation after Micro Branch to address this problem. By integrating predictions of a supervised model with predictions based on pixel brightness and edge detection, our system is less influenced by the quality of annotations. Experimental findings indicate that even with low quality annotations, the suggested framework keeps a good detection precision. 

### This implementation requires to download the repositories from clone the repository from "hitachi-rd-cv/weakly-sup-crackdet"


In [1]:
! git clone --recursive https://github.com/hitachi-rd-cv/weakly-sup-crackdet.git

Cloning into 'weakly-sup-crackdet'...
remote: Enumerating objects: 182, done.
remote: Counting objects: 100% (182/182), done.
remote: Compressing objects: 100% (128/128), done.
remote: Total 182 (delta 68), reused 141 (delta 40), pack-reused 0
Receiving objects: 100% (182/182), 108.44 KiB | 1.90 MiB/s, done.
Resolving deltas: 100% (68/68), done.
Submodule 'data/cfd_github' (https://github.com/cuilimeng/CrackForest-dataset) registered for path 'data/cfd_github'
Submodule 'data/deepcrack_github' (https://github.com/yhlleo/DeepCrack.git) registered for path 'data/deepcrack_github'
Submodule 'models/deepcrack' (http://github.com/yhlleo/DeepSegmentor) registered for path 'models/deepcrack'
Submodule 'models/deeplab' (http://github.com/tensorflow/models) registered for path 'models/deeplab'
Cloning into '/content/weakly-sup-crackdet/data/cfd_github'...
remote: Enumerating objects: 407, done.        
remote: Total 407 (delta 0), reused 0 (delta 0), pack-reused 407        
Receiving objects: 1

### Check if correct modules from repository have been downloaded. 

In [2]:
% cd /content/weakly-sup-crackdet/ # change directory and list the files to verify correct modules have been downloaded.
% ls -lrt

/content/weakly-sup-crackdet
total 44
-rwxr-xr-x 1 root root 9463 Aug 11 18:44 README.md*
drwxr-xr-x 2 root root 4096 Aug 11 18:44 docker/
drwxr-xr-x 7 root root 4096 Aug 11 18:44 data/
-rwxr-xr-x 1 root root 4327 Aug 11 18:44 tldr.md*
-rwxr-xr-x 1 root root  388 Aug 11 18:44 requirements.txt*
drwxr-xr-x 4 root root 4096 Aug 11 18:44 models/
drwxr-xr-x 3 root root 4096 Aug 11 18:44 eval/
drwxr-xr-x 3 root root 4096 Aug 11 18:44 tools/


### This script downloads the CFD dataset from website using git clone https://github.com/cuilimeng/CrackForest-dataset data/cfd_github

In [7]:
! ./tools/download.sh

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 27510    0 27510    0     0  30634      0 --:--:-- --:--:-- --:--:-- 30600
--2021-08-11 18:52:33--  https://www.irit.fr/~Sylvie.Chambon/TITS/IMAGES/AIGLE_RN/Im_GT_AIGLE_RN_C18aor.png
Resolving www.irit.fr (www.irit.fr)... 141.115.28.2
Connecting to www.irit.fr (www.irit.fr)|141.115.28.2|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 398766 (389K) [image/png]
Saving to: ‘data/aigle_github/original/Im_GT_AIGLE_RN_C18aor.png’

Im_GT_AIGLE_RN_C18a 100%[===================>] 389.42K   631KB/s    in 0.6s    

2021-08-11 18:52:35 (631 KB/s) - ‘data/aigle_github/original/Im_GT_AIGLE_RN_C18aor.png’ saved [398766/398766]

--2021-08-11 18:52:35--  https://www.irit.fr/~Sylvie.Chambon/TITS/IMAGES/AIGLE_RN/Im_GT_AIGLE_RN_F01aor.png
Reusing existing connection to www.irit.fr:443.
HTTP request sent, awaiting respons

### This script prepares the images in CFD dataset to be able to perfrom synthetic low annotation. 

In [8]:
! python tools/data_gen.py --fill

2021-08-11 18:53:31.604827: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [9]:
% cd /content/weakly-sup-crackdet
% ls -lrt

/content/weakly-sup-crackdet
total 44
-rwxr-xr-x 1 root root 9463 Aug 11 18:44 README.md*
drwxr-xr-x 2 root root 4096 Aug 11 18:44 docker/
-rwxr-xr-x 1 root root 4327 Aug 11 18:44 tldr.md*
-rwxr-xr-x 1 root root  388 Aug 11 18:44 requirements.txt*
drwxr-xr-x 4 root root 4096 Aug 11 18:44 models/
drwxr-xr-x 3 root root 4096 Aug 11 18:44 eval/
drwxr-xr-x 4 root root 4096 Aug 11 18:52 tools/
drwxr-xr-x 8 root root 4096 Aug 11 18:52 data/


### The below script performs synthetic low quality annotation of the images in CFD dataset where

*   --anno_type = dilation value (n_{dil} in the paper). 
*   --dataset_name = dataset name.

Larger value implies lower quality annotation. 

In [10]:
! python tools/data_gen.py --synth_dil_anno --anno_type 1 2 3 4 --dataset_name cfd

2021-08-11 18:53:39.380089: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
synthesizing data/cfd_1
001.jpg, precision:0.491, recall:0.929, alpha:104.000
002.jpg, precision:0.600, recall:0.971, alpha:62.000
003.jpg, precision:0.513, recall:0.944, alpha:79.000
004.jpg, precision:0.499, recall:0.943, alpha:114.000
005.jpg, precision:0.544, recall:0.930, alpha:71.000
006.jpg, precision:0.528, recall:0.970, alpha:66.000
007.jpg, precision:0.450, recall:0.969, alpha:57.000
008.jpg, precision:0.522, recall:0.955, alpha:84.000
009.jpg, precision:0.571, recall:0.957, alpha:75.000
010.jpg, precision:0.567, recall:0.966, alpha:69.000
011.jpg, precision:0.418, recall:0.926, alpha:100.000
012.jpg, precision:0.613, recall:0.960, alpha:89.000
013.jpg, precision:0.585, recall:0.956, alpha:92.000
014.jpg, precision:0.469, recall:0.925, alpha:86.000
015.jpg, precision:0.476, recall:0.958, alpha:59.000
016.jpg, precision:0.533, recall

### The below script builds DeepCrack network.

In [12]:
! ./tools/setup_models.sh --deepcrack  # Building  DeepCrack network 

### Detailed and dil1 annotations for DeepCrack format and dispatch

In [13]:
! python tools/data_gen.py --deepcrack --anno_type detailed 1 --dataset_name cfd

2021-08-11 19:04:09.999594: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
deepcrack data/cfd_detailed
deepcrack data/cfd_dil1


### Installing "dominate" package as it is not available by default.

In [14]:
pip install dominate

### Provide execution permissions to entire respository and change directory to deepcrack to start training the model.

In [16]:
! chmod -R 775 /content/weakly-sup-crackdet/
% cd /content/weakly-sup-crackdet/models/deepcrack/

/content/weakly-sup-crackdet/models/deepcrack


### The following script trains the deepcrack model with the following parameters. 

•	BATCH_SIZE=1

•	NUM_CLASSES=1

•	LOAD_WIDTH=256

•	LOAD_HEIGHT=256

•	The number of training images = 71

•	Total number of parameters : 14.720 M

•	lr = 1e-3 

•	epochs = 700 

•	wcmodel = 33.3

The results are saved in /content/weakly-sup-crackdet/models/deepcrack/checkpoints directory.

In [17]:
! scripts/train_deepcrack.sh

----------------- Options ---------------
               batch_size: 1                             
                    beta1: 0.5                           
          checkpoints_dir: ./checkpoints                 
           continue_train: False                         
                crop_size: 256                           
                 dataroot: ./datasets/aigle_dil1         	[default: None]
             dataset_mode: deepcrack                     
                direction: AtoB                          
              display_env: main                          
             display_freq: 400                           
               display_id: 0                             	[default: 1]
            display_ncols: 4                             
             display_port: 8123                          
           display_server: http://localhost              
            display_sides: 1                             
          display_winsize: 256                           
 

### To perform evaluation, it is required to copy the results from /models/deepcrack/checkpoints to /eval/results directory. 

In [24]:
% cd /content/weakly-sup-crackdet/models/deepcrack/checkpoints/
% ls -lrt
% cp -r * /content/weakly-sup-crackdet/eval/results

/content/weakly-sup-crackdet/models/deepcrack/checkpoints
total 12
drwxr-xr-x 3 root root 4096 Aug 11 19:51 aigle_deepcrack_dil1/
drwxr-xr-x 5 root root 4096 Aug 11 19:51 cfd_deepcrack_dil1/
drwxr-xr-x 3 root root 4096 Aug 11 19:51 dc_deepcrack_dil1/


### Change directory to execute evaluations. 

In [26]:
% cd /content/weakly-sup-crackdet/

/content/weakly-sup-crackdet


### In order to perform evaluations multiple times it is required to remove the previous evaluation results. For ex,consider evaluation for only Macro branch has been performed and next we need to check evaluations for Micro branch. In this case results from previous Macro branch needs to be deleted. 

In [62]:
# % rm -rf /content/weakly-sup-crackdet/eval/results/cfd_deepcrack_dil1/sample_imgs/cv_output_none
# % rm /content/weakly-sup-crackdet/eval/results/cfd_deepcrack_dil1/eval_cv_output_none_dil0.txt
# % rm /content/weakly-sup-crackdet/eval/results/cfd_deepcrack_dil1/eval_test_output_dil0.txt

### The following script perfroms the evaluations and saves the results in /eval/results/cfd_deepcrack_dil1/sample_imgs/cv_output_none path. The perfromance score details are stored in /eval/results/cfd_deepcrack_dil1/eval_cv_output_none_dil0.txt file. 

In [63]:
! python eval/micro_eval.py

applying cv models...
--------------------
dc_deepcrack_dil1
--------------------
test_output_gf
--------------------
test_output
--------------------
.gitkeep
--------------------
aigle_deepcrack_dil1
--------------------
cfd_deepcrack_dil1
processing eval/results/cfd_deepcrack_dil1/sample_imgs/test_output...
eval/results/cfd_deepcrack_dil1/sample_imgs/cv_output_none
evaluating the models
evaluating eval/results/cfd_deepcrack_dil1/sample_imgs/test_output...
evaluating eval/results/cfd_deepcrack_dil1/sample_imgs/test_output...
